In [2]:
using Lattices

# 1D Box index

In [33]:
function box_indices(ltc::Lattice1D, b)
    @assert ltc.N%b == 0
    L = ltc.N
    U = ltc.U
    box_pts = U*b 
    box_num = L÷b 
    inds = Array{Int64}(undef, box_pts, box_num)
    box_count=1 
    for x in 1:b:L-b+1
        for n in 0:b-1, u in 1:U 
            inds[U*n+u, box_count] = index(ltc, (x+n, u)) 
        end 
        box_count += 1
    end 
    return inds
end

box_indices (generic function with 4 methods)

In [5]:
ltc = Lattice1D(100, 2)
box_indices(ltc, 2)

4

4×50 Matrix{Int64}:
 1  5   9  13  17  21  25  29  33  37  …  173  177  181  185  189  193  197
 2  6  10  14  18  22  26  30  34  38     174  178  182  186  190  194  198
 3  7  11  15  19  23  27  31  35  39     175  179  183  187  191  195  199
 4  8  12  16  20  24  28  32  36  40     176  180  184  188  192  196  200

In [6]:
ltc = Lattice1D(99, 3)
box_indices(ltc, 9)

27

27×11 Matrix{Int64}:
  1  28  55   82  109  136  163  190  217  244  271
  2  29  56   83  110  137  164  191  218  245  272
  3  30  57   84  111  138  165  192  219  246  273
  4  31  58   85  112  139  166  193  220  247  274
  5  32  59   86  113  140  167  194  221  248  275
  6  33  60   87  114  141  168  195  222  249  276
  7  34  61   88  115  142  169  196  223  250  277
  8  35  62   89  116  143  170  197  224  251  278
  9  36  63   90  117  144  171  198  225  252  279
 10  37  64   91  118  145  172  199  226  253  280
 11  38  65   92  119  146  173  200  227  254  281
 12  39  66   93  120  147  174  201  228  255  282
 13  40  67   94  121  148  175  202  229  256  283
  ⋮                      ⋮                        ⋮
 16  43  70   97  124  151  178  205  232  259  286
 17  44  71   98  125  152  179  206  233  260  287
 18  45  72   99  126  153  180  207  234  261  288
 19  46  73  100  127  154  181  208  235  262  289
 20  47  74  101  128  155  182  209  236  

# 2D Box index

In [7]:
function box_indices(ltc::Lattice2D, b)
    @assert ltc.N%b == 0
    L = ltc.N
    U = ltc.U
    box_pts = U*b^2 
    box_num = L^2÷b^2 
    display(box_pts)
    inds = Array{Int64}(undef, box_pts, box_num)
    box_count=1 
    for x in 1:b:L-b+1, y in 1:b:L-b+1
        idx_count = 1
        for m in 0:b-1, n in 0:b-1, u in 1:U 
            inds[idx_count, box_count] = index(ltc, (x+m,y+n, u)) 
            idx_count += 1
        end 
        box_count += 1
    end 
    return inds
end

box_indices (generic function with 2 methods)

In [8]:
ltc = Lattice2D(10, 10, 1)
box_indices(ltc, 2)

4

4×25 Matrix{Int64}:
  1   3   5   7   9  21  23  25  27  …  63  65  67  69  81  83  85  87   89
  2   4   6   8  10  22  24  26  28     64  66  68  70  82  84  86  88   90
 11  13  15  17  19  31  33  35  37     73  75  77  79  91  93  95  97   99
 12  14  16  18  20  32  34  36  38     74  76  78  80  92  94  96  98  100

In [9]:
function box_indices(ltc::Lattice3D, b)
    @assert ltc.N%b == 0 && ltc.L == ltc.M == ltc.N
    L = ltc.N
    U = ltc.U
    box_pts = U*b^3
    box_num = L^3÷b^3 
    display(box_pts)
    inds = Array{Int64}(undef, box_pts, box_num)
    box_count=1 
    for x in 1:b:L-b+1, y in 1:b:L-b+1, z in 1:b:L-b+1
        idx_count = 1
        for l in 0:b-1, m in 0:b-1, n in 0:b-1, u in 1:U 
            inds[idx_count, box_count] = index(ltc, (x+l,y+m,z+n, u)) 
            idx_count += 1
        end 
        box_count += 1
    end 
    return inds
end

box_indices (generic function with 3 methods)

In [72]:
ltc = Lattice3D(100, 100, 100, 1)
@time box_indices(ltc, 2)

8

  0.012933 seconds (202 allocations: 7.643 MiB)


8×125000 Matrix{Int64}:
     1      3      5      7      9     11  …  989893  989895  989897   989899
     2      4      6      8     10     12     989894  989896  989898   989900
   101    103    105    107    109    111     989993  989995  989997   989999
   102    104    106    108    110    112     989994  989996  989998   990000
 10001  10003  10005  10007  10009  10011     999893  999895  999897   999899
 10002  10004  10006  10008  10010  10012  …  999894  999896  999898   999900
 10101  10103  10105  10107  10109  10111     999993  999995  999997   999999
 10102  10104  10106  10108  10110  10112     999994  999996  999998  1000000

In [60]:
function box_coarse(p, box_inds)
    @assert size(p, 1) == length(box_inds) 
    p_coarse  = Array{eltype(p)}(undef, size(box_inds, 2), size(p, 2)) 
    for j in 1:size(p, 2), i in 1:size(box_inds, 2)
        p_coarse[i, j] = sum(p[box_inds[:, i], j])
    end
    
    if typeof(p) <: Vector
        return vec(p)
    else
        return p_coarse
    end
end

box_coarse (generic function with 1 method)

In [67]:
ltc = Lattice2D(100, 100, 2)
box_inds = box_indices(ltc, 2)
x = rand(20000, 2)
@code_warntype box_coarse(x, box_inds)

8

MethodInstance for box_coarse(::Matrix{Float64}, ::Matrix{Int64})
  from box_coarse(p, box_inds) in Main at In[60]:1
Arguments
  #self#::Core.Const(box_coarse)
  p::Matrix{Float64}
  box_inds::Matrix{Int64}
Locals
  @_4::Union{Nothing, Tuple{Int64, Int64}}
  p_coarse::Matrix{Float64}
  @_6::Union{Nothing, Tuple{Int64, Int64}}
  j@_7::Int64
  i::Int64
  j@_9::Int64
Body::Matrix{Float64}
1 ──       Core.NewvarNode(:(@_4))
│          Core.NewvarNode(:(p_coarse))
│    %3  = Main.size(p, 1)::Int64
│    %4  = Main.length(box_inds)::Int64
│    %5  = (%3 == %4)::Bool
└───       goto #3 if not %5
2 ──       goto #4
3 ── %8  = Base.AssertionError("size(p, 1) == length(box_inds)")::Any
└───       Base.throw(%8)
4 ┄─ %10 = Main.eltype(p)::Core.Const(Float64)
│    %11 = Core.apply_type(Main.Array, %10)::Core.Const(Array{Float64})
│    %12 = Main.size(box_inds, 2)::Int64
│    %13 = Main.size(p, 2)::Int64
│          (p_coarse = (%11)(Main.undef, %12, %13))
│    %15 = Main.size(p, 2)::Int64
│    %16 =

In [55]:
sum(x[1:4])

0.9372018146777468